In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
 
# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
all_data_df = pd.read_csv("../input/test-dataset-for-titanic-competition/titanic_train.csv")
test_df = pd.read_csv("../input/test-dataset-for-titanic-competition/titanic_test.csv")
df = pd.read_csv("../input/test-dataset-for-titanic-competition/titanic_train.csv")
combine = [df, test_df]

In [ ]:
print(all_data_df.columns)
print("SPACE SPACE")
#print(all_data_df.describe(include=['object']))
print(all_data_df.head())

In [ ]:
my_features_df = all_data_df.drop(['PassengerId', 'Ticket', 'Cabin', 'Embarked'], axis=1)
test_df.drop(['PassengerId', 'Ticket', 'Cabin', 'Embarked'], axis=1, inplace=True)
survived = all_data_df['Survived']
print(my_features_df)
print("SPACE SPACE")
print(test_df)

In [ ]:
sexx = {'male': 1,'female': 0}
my_features_df['Sex'] = my_features_df['Sex'].apply(lambda x: sexx[x])
test_df['Sex'] = test_df['Sex'].apply(lambda x: sexx[x])
print(my_features_df)

In [ ]:
age_mean = my_features_df['Age'].mean()
age_mean_test = test_df['Age'].mean()
my_features_df['Age'].fillna(age_mean, inplace=True)
test_df['Age'].fillna(age_mean_test, inplace=True)
print("space SPACE")
print(my_features_df['Age'].isnull().values.any())
print(test_df['Age'].isnull().values.any())

In [ ]:
Pclass_divide_Fare = my_features_df['Pclass'] / my_features_df['Fare']
Pclass_divide_Fare_test = test_df['Pclass'] / test_df['Fare']
print(Pclass_divide_Fare)

In [ ]:
my_features_df.loc[(my_features_df.Pclass == 1) & (my_features_df.Fare > 50), 'clss1_fareGt50'] = 1
my_features_df['clss1_fareGt50'].fillna(0, inplace=True)

my_features_df.loc[(my_features_df.Pclass == 2) & (my_features_df.Fare < 50) & (my_features_df.Fare > 30), 'clss2_fareGt30_fareLt50'] = 1
my_features_df['clss2_fareGt30_fareLt50'].fillna(0, inplace=True)

my_features_df.loc[(my_features_df.Pclass == 3) & (my_features_df.Fare < 30), 'clss3_fareLt30'] = 1
my_features_df['clss3_fareLt30'].fillna(0, inplace=True)
print(my_features_df)




test_df.loc[(test_df.Pclass == 1) & (test_df.Fare > 50), 'clss1_fareGt50'] = 1
test_df['clss1_fareGt50'].fillna(0, inplace=True)

test_df.loc[(test_df.Pclass == 2) & (test_df.Fare < 50) & (test_df.Fare > 30), 'clss2_fareGt30_fareLt50'] = 1
test_df['clss2_fareGt30_fareLt50'].fillna(0, inplace=True)

test_df.loc[(test_df.Pclass == 3) & (test_df.Fare < 30), 'clss3_fareLt30'] = 1
test_df['clss3_fareLt30'].fillna(0, inplace=True)
print(my_features_df)
print("SPACE SPACE")
print(test_df)

In [ ]:
my_features_df['devide'] = Pclass_divide_Fare
print(my_features_df)

test_df['devide'] = Pclass_divide_Fare_test
print("SPACE SPACE SPACE")
print(test_df)

In [ ]:
print(my_features_df)

In [ ]:
for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
    
pd.crosstab(df['Title'], df['Survived'])

In [ ]:
for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
 	'Dr', 'Major', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    dataset['Title'] = dataset['Title'].replace('Don', 'Mr')
    dataset['Title'] = dataset['Title'].replace('Rav', 'Mr')
    
df[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

In [ ]:
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)
    
df.head()

In [ ]:
title_feature = df['Title']
my_features_df['Title'] = title_feature
print(my_features_df)

test_df['Title'] = title_feature

In [ ]:
my_features_df

In [ ]:
survived = my_features_df[my_features_df.Survived == 1]
unsurvived = my_features_df[my_features_df.Survived == 0]


corr = my_features_df.corr(method ='pearson')
corr_survived= survived.corr(method ='pearson')
corr_unsurvived= unsurvived.corr(method ='pearson')
#corr1 =  pd.concat([corr_survived, corr_unsurvived] )
#corr2 = corr/corr_survived

In [ ]:
ax = sns.heatmap(corr,cmap='coolwarm',     annot=True,     annot_kws={'size':8})

In [ ]:
my_features_df.drop(['Name'], axis=1, inplace=True)
test_df.drop(['Name'], axis=1, inplace=True)

In [ ]:
X_train= my_features_df.drop('Survived', axis=1)
Y_train= my_features_df['Survived']
X_test= test_df.copy()

In [ ]:
print(X_test.devide)

In [ ]:
X_test = X_test.fillna(0)
X_test.drop(['devide'], axis=1, inplace=True)
X_train.drop(['devide'], axis=1, inplace=True)

In [ ]:
print(X_test.describe())

In [ ]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)

# Y_pred = random_forest.predict(X_test)
# random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_test, Y_train) * 100, 2)
acc_random_forest